# 🧹 Finance ABAC Demo - Step 5: Cleanup

## 📋 Overview
This notebook cleans up all resources created during the Finance ABAC demo, including policies, tags, tables, and schemas.

### What This Notebook Does:
1. **Drops ABAC Policies**: Removes all schema-level policies
2. **Deletes Tag Policies**: Removes account-level tag policies
3. **Drops Schema**: Removes the schema and all its tables

### ⚠️ Warning:
**This notebook will permanently delete:**
- All ABAC policies on the finance schema
- All finance-specific tag policies
- The entire finance schema and all its tables
- All data in those tables

**Only run this if you want to completely remove the demo!**

## 🎓 How to Use This Notebook
1. **Ensure You Want to Clean Up**: This is destructive!
2. **Run All Cells**: Execute sequentially
3. **Verify Cleanup**: Check that resources are removed

## ⚙️ Prerequisites
- Admin permissions to drop policies and schemas
- Completed previous notebooks (or at least have resources to clean)

---


In [ ]:
pip install pyyaml


In [ ]:
# 📋 Load Configuration from config.yaml
import yaml
from pathlib import Path

config_file = Path('config.yaml')
if config_file.exists():
    with open(config_file) as f:
        config = yaml.safe_load(f)
    CATALOG = config['catalog']
    SCHEMA = config['schema']
    print(f'✅ Configuration loaded from config.yaml')
    print(f'   📊 Catalog: {CATALOG}')
    print(f'   📁 Schema: {SCHEMA}')
else:
    # Fallback defaults
    CATALOG = 'your_catalog_name'
    SCHEMA = 'finance'
    print(f'⚠️  config.yaml not found - using defaults')
    print(f'   📊 Catalog: {CATALOG}')
    print(f'   📁 Schema: {SCHEMA}')

# Set catalog and schema
spark.sql(f"USE CATALOG {CATALOG}")
spark.sql(f"USE SCHEMA {SCHEMA}")


## Step 1: Drop All ABAC Policies


In [ ]:
try:
    policies = (spark.sql(f"SHOW POLICIES ON SCHEMA {SCHEMA}")).collect()
    for policy in policies:
        policy_name = policy["Policy Name"]
        try:
            spark.sql(f"DROP POLICY {policy_name} ON SCHEMA {SCHEMA}")
            print(f"✅ Dropped policy: {policy_name}")
        except Exception as e:
            print(f"⚠️  Could not drop {policy_name}: {str(e)[:40]}")
    print(f"\n✅ All policies dropped successfully")
except Exception as e:
    print(f"✅ No policies found or already cleaned up")


## Step 2: Delete Tag Policies


In [ ]:
import requests
from databricks.sdk import WorkspaceClient

client = WorkspaceClient()
workspace_url = client.config.host

def get_token():
    ctx = dbutils.notebook.entry_point.getDbutils().notebook().getContext()
    return getattr(ctx, "apiToken")().get()

def delete_tag_policy(tag_key):
    """Delete a tag policy by key"""
    try:
        data = requests.delete(
            f"{workspace_url}/api/2.1/tag-policies/{tag_key}",
            headers={"Authorization": f"Bearer {get_token()}"}
        )
        if data.status_code == 200:
            print(f"✅ Deleted tag policy: {tag_key}")
        elif data.status_code == 404:
            print(f"ℹ️  Tag policy not found: {tag_key}")
        else:
            print(f"⚠️  Could not delete {tag_key}: {data.text}")
        return data
    except Exception as e:
        print(f"⚠️  Error deleting {tag_key}: {str(e)}")
        return None


In [ ]:
# Delete all finance-specific tag policies
delete_tag_policy('pii_type_finance')
delete_tag_policy('pci_compliance_finance')
delete_tag_policy('data_classification_finance')
delete_tag_policy('fraud_detection_finance')

print("\n✅ Tag policy cleanup complete")


## Step 3: Drop Schema and All Tables


In [ ]:
spark.sql(f"DROP SCHEMA IF EXISTS {SCHEMA} CASCADE")
print(f"✅ Dropped schema: {CATALOG}.{SCHEMA}")


## ✅ Cleanup Complete!

All Finance ABAC demo resources have been removed:

### What Was Deleted:
- ✅ All ABAC policies (column masks and row filters)
- ✅ All finance-specific tag policies
- ✅ Finance schema and all tables
- ✅ All sample data

### What Remains:
- ✅ Catalog (not deleted - may be shared across multiple schemas)

### 🎯 Next Steps:

If you want to run the demo again:
1. Start from **`1_Create_Functions.ipynb`**
2. Run all notebooks in sequence
3. Everything will be recreated fresh

---
**🎉 Thank you for exploring Finance ABAC!**
